In [2]:
import sys
sys.path.append('./../../')
sys.path.append('./..')
sys.path.append('./../../designer/script/')


from film import FreeFormFilm, BaseFilm, TwoMaterialFilm, EqOTFilm
from design import BaseDesign
from spectrum import BaseSpectrum, Spectrum
from optimizer.adam import AdamFreeFormOptimizer

from analyze_utils.make_design import make_edgefilter_design, make_reflection_design, make_three_line_filter_design
from utils.loss import calculate_RMS_f_spec
from analyze_utils.result_io import save, load
from analyze_utils.structure import plot_layer_thickness

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import copy
import pickle

In [3]:
# impact of hyperpram on convergence
target = make_three_line_filter_design().target_specs
max_step = 100

def exp(layer_ot, n_size):
    n = np.random.random(n_size) + 1.5 # 1.5 to 2.5, init ot dif from final ot by: 1/4
    film = EqOTFilm(n, layer_ot * n_size, 'SiO2')
    
    optimizer = AdamFreeFormOptimizer(
        film, 
        target, 
        max_steps=max_step, 
        alpha=0.1, 
        record=True, 
        show=False, 
        wl_batch_size=200
    )
    losses, films = optimizer()
    return losses


layer_ot = 1
n_size = 1000

fig, ax = plt.subplots(1, 1)
ax.plot(np.arange(max_step), exp(layer_ot, n_size))

